# Bizhub Internal API Tester

这个 Notebook 用于测试 Bizhub 的内部 API 接口，包含了自动签名和 AES-GCM 加密功能。

In [7]:
# 1. 配置环境和依赖
import os
import json
import time
import uuid
import hmac
import hashlib
import base64
import requests
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

# ================= 配置区 =================
# 请在此处填入你的 Key，或者确保它们在环境变量中
BIZHUB_SECRET_KEY = os.environ.get('BIZHUB_SECRET_KEY', 'vyQVTdia3SmiT0FfuHMEmds64Q86zW-9M9LGSxgzgS9sYJUQqWac_WHQ8tm42f1I')
BIZHUB_AES_KEY = os.environ.get('BIZHUB_AES_KEY', '60de7302c514a30b83d659ea1643e9b5')

# API 基础 URL (请根据实际情况修改，例如本地调试用 localhost:3000)
BASE_URL = "http://localhost:3000"
# =========================================

print(f"Target Base URL: {BASE_URL}")
print(f"Secret Key set: {bool(BIZHUB_SECRET_KEY)}")
print(f"AES Key set: {bool(BIZHUB_AES_KEY)}")

Target Base URL: http://localhost:3000
Secret Key set: True
AES Key set: True


In [8]:
# 2. 定义加密和签名函数

def generate_signature(timestamp, nonce, body_for_signature, secret_key):
    """
    生成 HMAC-SHA256 签名
    """
    sign_string = f"{timestamp}\n{nonce}\n{body_for_signature}"
    signature = hmac.new(
        secret_key.encode('utf-8'),
        sign_string.encode('utf-8'),
        hashlib.sha256
    ).hexdigest()
    return signature

def encrypt_aes_gcm(payload_dict, aes_key_str):
    """
    使用 AES-256-GCM 加密 payload
    返回 base64(IV(16) + Tag(16) + Ciphertext)
    """
    key_bytes = aes_key_str.encode('utf-8')
    if len(key_bytes) != 32:
        raise ValueError(f"AES Key must be exactly 32 bytes. Current: {len(key_bytes)}")

    aesgcm = AESGCM(key_bytes)
    iv = os.urandom(16)
    
    # 序列化 JSON，保持 key 排序以一致性（非强制，但推荐）
    payload_str = json.dumps(payload_dict, sort_keys=True, separators=(',', ':'))
    data = payload_str.encode('utf-8')
    
    # encrypt() 返回 ciphertext + tag
    ct_and_tag = aesgcm.encrypt(iv, data, None)
    
    # 提取 tag 和 ciphertext
    tag = ct_and_tag[-16:]
    ciphertext = ct_and_tag[:-16]
    
    encrypted_bytes = iv + tag + ciphertext
    return base64.b64encode(encrypted_bytes).decode('utf-8')

def send_request(method, endpoint, payload=None, encrypt=False):
    """
    发送带有签名和加密的请求
    """
    url = f"{BASE_URL}{endpoint}"
    timestamp = str(int(time.time() * 1000))
    nonce = uuid.uuid4().hex
    
    final_body_str = ""
    body_for_signature = ""
    
    if method == 'GET':
        final_body_str = "" # GET 无 body
        body_for_signature = ""
    else:
        if encrypt:
            if not BIZHUB_AES_KEY:
                raise ValueError("Encryption requested but BIZHUB_AES_KEY is missing")
            
            # 加密
            encrypted_data = encrypt_aes_gcm(payload, BIZHUB_AES_KEY)
            final_body_dict = {"encrypted_data": encrypted_data}
            final_body_str = json.dumps(final_body_dict)
            body_for_signature = encrypted_data
        else:
            # 明文
            final_body_str = json.dumps(payload, separators=(',', ':')) if payload else ""
            body_for_signature = final_body_str

    # 生成签名
    signature = generate_signature(timestamp, nonce, body_for_signature, BIZHUB_SECRET_KEY)
    
    headers = {
        "X-Timestamp": timestamp,
        "X-Nonce": nonce,
        "X-Signature": signature,
        "Content-Type": "application/json",
        "User-Agent": "Bizhub-Test-Client/1.0"
    }
    
    print(f"Sending {method} to {url}...")
    start_time = time.time()
    
    response = requests.request(method, url, data=final_body_str, headers=headers)
    
    duration = (time.time() - start_time) * 1000
    print(f"Status: {response.status_code} ({duration:.0f}ms)")
    
    try:
        return response.json()
    except:
        return response.text

### 测试 1: 同步用户 (POST + 加密)

In [9]:
payload = {
    "bizhub_user_id": "test_bizhub_user_001",
    "email": "teser_001@example.com",
    "username": "测试用户001",
    "phone": "13800138000",
    "tk_saas_user_id": "tksaas_user_90001"
}

# 注意：encrypt=True 开启加密
res_sync = send_request("POST", "/api/v1/internal/sync-user", payload, encrypt=True)
print(json.dumps(res_sync, indent=2, ensure_ascii=False))

Sending POST to http://localhost:3000/api/v1/internal/sync-user...
Status: 200 (2658ms)
{
  "code": 200,
  "msg": "用户同步成功",
  "data": {
    "tk_saas_user_id": "tksaas_user_90001",
    "is_new": false,
    "synced": true
  }
}


### 测试 2: 查询子账户 (GET + 明文)

In [10]:
# GET 请求不需要 payload 和 encrypt 参数
# 假设这是上面创建的用户的 ID (你需要根据实际数据库里生成的 ID 替换)
# 如果你还不知道 ID，可以先去数据库查一下，或者通过上面的 sync 接口返回（如果它返回了 ID）

# 这里使用一个示例 ID，请替换为真实存在的 parent_user_id
parent_user_id = "test_bizhub_user_001" 

endpoint = f"/api/v1/internal/user/children?parent_user_id={parent_user_id}"
res_children = send_request("GET", endpoint)
print(json.dumps(res_children, indent=2, ensure_ascii=False))

Sending GET to http://localhost:3000/api/v1/internal/user/children?parent_user_id=test_bizhub_user_001...
Status: 200 (439ms)
{
  "code": 200,
  "msg": "查询成功",
  "data": {
    "parent_user_id": "test_bizhub_user_001",
    "children": []
  }
}


### 测试 3: 获取 Ticket (POST + 明文 - 如果配置允许)

In [11]:
payload_ticket = {
    "user_id": "test_bizhub_user_001",
    "source": "web"
}

# 假设 get-ticket 接口在白名单里，可以是明文
res_ticket = send_request("POST", "/api/v1/internal/get-ticket", payload_ticket, encrypt=False)
print(json.dumps(res_ticket, indent=2, ensure_ascii=False))

Sending POST to http://localhost:3000/api/v1/internal/get-ticket...
Status: 400 (25ms)
{
  "code": 400,
  "msg": "[\n  {\n    \"code\": \"custom\",\n    \"path\": [],\n    \"message\": \"需要至少提供一个用户标识：bizhub_user_id | tk_saas_user_id | email | phone\"\n  }\n]",
  "data": null
}


In [12]:
# 测试 4: 查询会员权益 (GET + 明文)
# 需要你替换成真实存在的 user_id
user_id = "5AXauIuIWyx6Kzo2y4kt8YHRndurFmql"

endpoint = f"/api/v1/internal/membership/rights?user_id={user_id}"
res_rights = send_request("GET", endpoint)
print(json.dumps(res_rights, indent=2, ensure_ascii=False))



Sending GET to http://localhost:3000/api/v1/internal/membership/rights?user_id=5AXauIuIWyx6Kzo2y4kt8YHRndurFmql...
Status: 200 (4620ms)
{
  "code": 200,
  "msg": "查询成功",
  "data": {
    "user_id": "5AXauIuIWyx6Kzo2y4kt8YHRndurFmql",
    "account": {
      "role": "parent",
      "is_parent": true,
      "is_child": false,
      "parent_user_id": null,
      "relationship": null,
      "children_count": 2
    },
    "membership": {
      "source": "subscription",
      "membership_code": "personal",
      "tier": {
        "code": "personal",
        "name": "个人版",
        "level": 2,
        "discount_rate": 100,
        "disabled": false
      },
      "plan": {
        "id": "personal",
        "is_free": false,
        "is_lifetime": false
      },
      "subscription": {
        "status": "active",
        "interval": "year",
        "period_start": 1765078777000,
        "period_end": 1796614777000,
        "cancel_at_period_end": false,
        "trial_start": null,
        "trial

In [13]:
# 测试 5: 查询积分流水 (GET + 明文)
# 需要你替换成真实存在的 user_id
user_id = "5AXauIuIWyx6Kzo2y4kt8YHRndurFmql"
page = 1
page_size = 20

endpoint = f"/api/v1/internal/voucher/list?user_id={user_id}&page={page}&page_size={page_size}"
# 可选：按 type 过滤
# endpoint += "&type=USAGE"

res_vouchers = send_request("GET", endpoint)
print(json.dumps(res_vouchers, indent=2, ensure_ascii=False))



Sending GET to http://localhost:3000/api/v1/internal/voucher/list?user_id=5AXauIuIWyx6Kzo2y4kt8YHRndurFmql&page=1&page_size=20...
Status: 200 (2687ms)
{
  "code": 200,
  "msg": "查询成功",
  "data": {
    "user_id": "5AXauIuIWyx6Kzo2y4kt8YHRndurFmql",
    "page": 1,
    "page_size": 20,
    "total": 8,
    "items": [
      {
        "id": "44029697-cc9a-4ba4-bcc4-ddf72996401a",
        "type": "PURCHASE_PACKAGE",
        "description": "+200 credits for package standard ($1,490)",
        "amount": 200,
        "remaining_amount": 200,
        "payment_id": "in_1SbiHDK5aoQEsLHScfrNXnuM",
        "expiration_date": 1767707168717,
        "created_at": 1765115168717,
        "updated_at": 1765115168717
      },
      {
        "id": "128816cf-43a2-4d88-97f2-96b6611dcff8",
        "type": "PURCHASE_PACKAGE",
        "description": "+200 credits for package standard ($1,490)",
        "amount": 200,
        "remaining_amount": 200,
        "payment_id": "in_1SbiHDK5aoQEsLHScfrNXnuM",
        "

In [14]:
# 测试 6: 积分使用通知 (POST + 加密)
# 需要你替换成真实存在的 user_id（bizhub_user_id）
payload_use = {
    "bizhub_user_id": "5AXauIuIWyx6Kzo2y4kt8YHRndurFmql",
    "order_id": "sample_order_10001",
    "amount": 5,
    "scene": "sample_grant",
    "description": "样品发放扣减积分 - sample_order_10001",
    "occurred_at": int(time.time() * 1000),
}

res_use = send_request("POST", "/api/v1/internal/voucher/use", payload_use, encrypt=True)
print(json.dumps(res_use, indent=2, ensure_ascii=False))



Sending POST to http://localhost:3000/api/v1/internal/voucher/use...
Status: 200 (4999ms)
{
  "code": 200,
  "msg": "扣减成功",
  "data": {
    "user_id": "5AXauIuIWyx6Kzo2y4kt8YHRndurFmql",
    "order_id": "sample_order_10001",
    "payment_id": "sample_order:sample_order_10001",
    "amount": 5,
    "idempotent": false,
    "transaction_id": "cac198e0-21d1-466f-a853-ec48514c2b08",
    "deducted": 5,
    "balance": 1265,
    "occurred_at": 1765627836481,
    "created_at": 1765627836481
  }
}


In [16]:
# 测试 7: 查询用户信息 (GET + 明文)
# 需要你替换成真实存在的 user_id
user_id = "5AXauIuIWyx6Kzo2y4kt8YHRndurFmql"

endpoint = f"/api/v1/internal/user/{user_id}"
res_user = send_request("GET", endpoint)
print(json.dumps(res_user, indent=2, ensure_ascii=False))



Sending GET to http://localhost:3000/api/v1/internal/user/5AXauIuIWyx6Kzo2y4kt8YHRndurFmql...
Status: 200 (454ms)
{
  "code": 200,
  "msg": "查询成功",
  "data": {
    "id": "5AXauIuIWyx6Kzo2y4kt8YHRndurFmql",
    "name": "18611697717",
    "email": "18611697717@mksaas.local",
    "emailVerified": false,
    "phoneNumber": "18611697717",
    "phoneNumberVerified": true,
    "role": "admin",
    "banned": null,
    "banReason": null,
    "banExpires": null,
    "customerId": "cus_TUGAqlY4Wc0qUg",
    "tkSaasUserId": "bz_test_1764599401",
    "synced": false,
    "createdAt": "2025-11-21T09:00:05.016Z",
    "updatedAt": "2025-11-25T08:30:21.219Z"
  }
}


In [ ]:
# 测试 8: 更新用户信息 (PUT + 加密)
# 需要你替换成真实存在的 user_id（bizhub_user_id）
payload_update = {
    "bizhub_user_id": "5AXauIuIWyx6Kzo2y4kt8YHRndurFmql",
    "username": "更新后的用户名",
    "email": "updated_email@example.com",
    "phone": "13900139000",
    "tk_saas_user_id": "updated_tksaas_001",
    "synced": True,
}

# 注意：PUT 请求也需要加密
res_update = send_request("PUT", "/api/v1/internal/user/update", payload_update, encrypt=True)
print(json.dumps(res_update, indent=2, ensure_ascii=False))

